In [ ]:
import numpy as np
import pandas as pd 
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import train_test_split

from sklearn import metrics

from sklearn import svm
import json



column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'target']

df = pd.read_csv('housing.csv', header=None, delimiter=r"\s+", names=column_names)

# target is MEDV
x = df.iloc[:,:-1]
y = df.iloc[:,-1]

mms= MinMaxScaler()
x = pd.DataFrame(mms.fit_transform(x), columns=x.columns)


xtrain,xtest,ytrain,ytest= train_test_split(x,y,test_size=0.3,random_state=42)

In [ ]:
# xtrain

In [ ]:
housing_train = np.column_stack((x, y))
# Append y_test to x_test as a column
housing_test = np.column_stack((xtest, ytest))

pd.DataFrame(housing_train, columns=column_names).to_csv('housing_train.csv', index=False)
pd.DataFrame(housing_test, columns=column_names).to_csv('housing_test.csv', index=False)


In [ ]:
import matplotlib.pyplot as plt


train = pd.read_csv('housing_train.csv')
test = pd.read_csv('housing_test.csv')


# Specify the columns to include
selected_features = ['LSTAT', 'CRIM', 'RM', 'PTRATIO']

# Subset the DataFrame to include only the selected features
X_train = train[selected_features]
X_train = train.drop(columns=['target'])
y_train = train['target']  # Assuming the target variable is 'MEDV'

# Feature names
feature_names = X_train.columns

# Fit a linear regression model
reg = LinearRegression()
reg.fit(X_train, y_train)

# Get the coefficients and their corresponding feature names
coefficients = reg.coef_
coeff_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': coefficients
})

# Sort the coefficients by absolute value in descending order
coeff_df['Absolute Coefficient'] = np.abs(coeff_df['Coefficient'])
coeff_df_sorted = coeff_df.sort_values(by='Absolute Coefficient', ascending=False)

plt.figure(figsize=(9, 6))
plt.bar(coeff_df_sorted['Feature'], coeff_df_sorted['Coefficient'])
plt.xlabel('Features', fontsize=14)  # Increase the fontsize to make the x-labels bigger
plt.ylabel('Coefficient', fontsize=14)
plt.title('Linear Regression Coefficients \n (Ordered by Absolute Value)', fontsize=16)
plt.xticks(rotation=90, fontsize=13)
plt.yticks(fontsize=13)
plt.show()

coeff_df_sorted
abs_coeff_sum = coeff_df_sorted['Absolute Coefficient'].sum()
abs_coeff_sum




In [ ]:
X_train.corr()

In [ ]:
input_ls = []

# count = 0
# for i in range(3):
#     for j in range(3):
#         if (i != j):
#             A = i
#             B = j
#             input = [("None", A, B, "clustered", "uniform", "lin_reg","bos_housing", "HOUSING_1.csv", "columns")]*20
#             input_ls.extend([*input])
#             count += 20

# print(count)


input = [("None", 0, 1, "clustered", "uniform", "lin_reg","bos_housing", "HOUSING_ALLTRAINLINREG.csv", "columns")]*10
input_ls.extend([*input])
input = [("None", 1, 0, "clustered", "uniform", "lin_reg","bos_housing", "HOUSING_ALLTRAINLINREG.csv", "columns")]*10
input_ls.extend([*input])
input = [("None", 1, 2, "clustered", "uniform", "lin_reg","bos_housing", "HOUSING_ALLTRAINLINREG.csv", "columns")]*10
input_ls.extend([*input])
input = [("None", 2, 1, "clustered", "uniform", "lin_reg","bos_housing", "HOUSING_ALLTRAINLINREG.csv", "columns")]*10
input_ls.extend([*input])
input = [("None", 0, 1, "clustered", "uniform", "random_forest_reg","bos_housing", "HOUSING_ALLTRAINRF.csv", "columns")]*10
input_ls.extend([*input])
input = [("None", 1, 0, "clustered", "uniform", "random_forest_reg","bos_housing", "HOUSING_ALLTRAINRF.csv", "columns")]*10
input_ls.extend([*input])
input = [("None", 1, 2, "clustered", "uniform", "random_forest_reg","bos_housing", "HOUSING_ALLTRAINRF.csv", "columns")]*10
input_ls.extend([*input])
input = [("None", 2, 1, "clustered", "uniform", "random_forest_reg","bos_housing", "HOUSING_ALLTRAINRF.csv", "columns")]*10
input_ls.extend([*input])



parameter_dict = {i: input_ls for i, input_ls in enumerate(input_ls)}

# Write the dictionary to a JSON file
with open("parameters_housing.json", "w") as json_file:
    json.dump(parameter_dict, json_file, indent=4)



In [ ]:

import numpy as np
import time
import pandas as pd
from collections.abc import Iterable
import itertools
from itertools import chain, combinations
from array import array
import copy
import random
import scipy.stats as st
from scipy.stats import norm
import math
import statistics
from collections import Counter
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from scipy.stats import multivariate_normal
from sklearn.neighbors import KernelDensity
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error
import csv
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn import metrics

def findShapleyMC(target_data, test_data, A, owners, n):
    
    
    permutations_dict = {}
    # A_data = owners[A]

    est_shap = 0
    for i in range(n):
        # draw a permutation, get coalition from permutation
        permutation = np.random.permutation(list(owners.keys()))
        S = findCoalitionOne(permutation, A)
        permString = listToString(S)

        P_data = []

        # get data from coalition
        if permString not in permutations_dict:
            P_data = [item for key in S for item in owners.get(key, [])]
            permutations_dict.update({permString: P_data})
        else:
            P_data = permutations_dict[permString]

        # print(P_data)

        # transfer data ownership between A and B
        # owners.update({A: list(np.append(owners[A], P_data))})

        # get utility of the data transfer
        if (len(P_data)==0):
            est_shap += 1/n*(utilityFunc(target_data, target_data, test_data, owners[A], "ha")+1)
        else:
            est_shap += 1/n*(utilityFunc(target_data, target_data, test_data, list(np.append(owners[A], P_data)), "ha")-utilityFunc(target_data, target_data, test_data, P_data, "ha"))
            # print((utilityLR(target_data, test_data, list(np.append(owners[A], P_data)))-utilityLR(target_data, test_data, P_data)))

        # owners.update({A: A_data})


    return est_shap

def utilityKDE(target_data, test_data, data_owner_data):

    data_owner_data = [int(x) for x in data_owner_data]

    feature = 11
    data = target_data.loc[data_owner_data]
    data_column = data.iloc[:, feature].values
    data_column = [x for x in data_column]
    
    target_column = target_data.iloc[:,feature].values
    target_column = [x for x in target_column]
    
    test_column = test_data.iloc[:,feature].values

    ground_truth_grid, ground_truth_density = kernel_density_estimation(target_column)

    sample_estimation_grid, sample_estimation_density = kernel_density_estimation(data_column)

    errors = []
    for data_item in test_column:
        gt_density = np.interp(data_item, ground_truth_grid, ground_truth_density)
        errors.append(abs(gt_density-np.interp(data_item, sample_estimation_grid, sample_estimation_density))/gt_density)

    return -np.mean(errors)

def findCoalitionOne(permutation, o):
    coa = []
    for data_owner in permutation:
        if (data_owner!=o):
            coa.append(data_owner)
        else:
            return sorted(coa)
    return sorted(coa)

def listToString(perm):
    return ' '.join(str(x) for x in perm)

def utilityLR(target_data, test_data, data_owner_data):
    if (len(data_owner_data)==0):
        return 0
    data_owner_data = [int(x) for x in data_owner_data]

    data = target_data.loc[data_owner_data]
    #print(data)

    data_features = data.drop("target", axis=1).values

    data_labels = data["target"].values
    test_data_features = test_data.drop("target", axis=1).values 

    test_data_labels = test_data["target"].values
    

    if (len(set(data_labels))==1):
        if (data_labels[0]==1):
            small_pred_probs = np.array([[0,1] for i in range(len(test_data_labels))])
        else:
            small_pred_probs = np.array([[1,0] for i in range(len(test_data_labels))])
    else: 
        small_logistic_regression = LogisticRegression(max_iter=1000,solver="liblinear",random_state=19)
        small_logistic_regression.fit(data_features, data_labels)
        small_pred_probs = small_logistic_regression.predict_proba(test_data_features)

    small_loss = 20-1*log_loss(test_data_labels, small_pred_probs)
    return small_loss

def utilityRF(target_data, test_data, data_owner_data):

    data_owner_data = [int(x) for x in data_owner_data]
    data = target_data.loc[data_owner_data]
    data_features = data.drop("target", axis=1).values
    data_labels = data["target"].values
    test_data_features = test_data.drop("target", axis=1).values


    test_data_labels = test_data["target"].values

    
    # Create and train the random forest classifier
    small_random_forest = RandomForestClassifier()
    small_random_forest.fit(data_features, data_labels)
    
    # Predict probabilities for the test data
    small_pred_probs = small_random_forest.predict(test_data_features)
    # prediction_classes = small_pred_probs.classes_

    # final_pred_probs = np.zeros((len(test_data), len(set(test_data_labels))))
    
    # Calculate the log loss
    small_loss = accuracy_score(small_pred_probs, test_data_labels)
    return small_loss

def assignClusters(df, cat):
    unique_categories = df[cat].unique()

    # Create a dictionary to store the lists of indices
    owner_data = {}
    category_indices = {}

    # Iterate through unique values and extract the indices
    for i in range(len(unique_categories)):
        category = unique_categories[i]
        owner_data[i] = df.index[df[cat] == category].tolist()
        category_indices[i] = category

    print(owner_data)
    return owner_data, category_indices

def kernel_density_estimation(data, bandwidth=0.2, kernel='gaussian', x_grid=None):
    # Fit the CF model
    data = np.array(data)
    kde = KernelDensity(bandwidth=bandwidth, kernel=kernel)
    kde.fit(data[:,np.newaxis])

    # If x_grid is not provided, create a default grid of values
    if x_grid is None:
        x_grid = np.linspace(data.min(), data.max(), 1000)

    # Compute the log-density values for the grid of values
    log_dens = kde.score_samples(x_grid[:,np.newaxis])

    # Exponentiate the log-densities to obtain the density values
    density = np.exp(log_dens)

    return x_grid, density

In [ ]:
import itertools
import math

def flatten(lst): return [item for sublist in lst for item in sublist]

def powerset(iterable): 
    s = list(iterable) 
    return itertools.chain.from_iterable(itertools.combinations(s, r) for r in range(len(s)+1))

def shapley(target_data, test, owners, A):
    A_data = owners[A]
    # B_data = owners[B]
    list_owners = list(owners.keys())
    list_owners.remove(A)
    # list_owners.remove(B)
    coalitions = powerset(list_owners)
    total = 0

    # go through all the powersets in order of size
    for S in coalitions:
        P_data = []
        for o in S:
            P_data.append(owners[o])
        P_data = list(flatten(P_data))
        owners.update({A: list(np.append(owners[A], P_data))})
        # owners.update({B: list(np.append(owners[B], P_data))})
        P_util = utilityFunc(target_data, target_data, test, P_data, "ha") if len(P_data) != 0 else 0
        addend = 1/(math.comb(len(list(owners.keys()))-1,len(S))) * (utilityFunc(target_data, target_data, test, owners[A], "ha") - P_util)
        # print("utility A and P", utilityFunc(target_data, target_data, test, owners[A], "ha"))
        # print("utility P", P_util)
        total += addend
        owners.update({A: A_data})
        # owners.update({B: B_data})
    return total/len(owners)

def diffShapleyCF(all_train_data, target_data, test, owners, A, B, util):
    A_data = owners[A]
    B_data = owners[B]
    list_owners = list(owners.keys())
    list_owners.remove(A)
    list_owners.remove(B)
    coalitions = powerset(list_owners)
    total = 0

    # go through all the powersets in order of size
    for S in coalitions:
        P_data = []
        for o in S:
            P_data.append(owners[o])
        P_data = list(flatten(P_data))
        owners.update({A: list(np.append(owners[A], P_data))})
        owners.update({B: list(np.append(owners[B], P_data))})
        total += 1/((len(S)+1)*math.comb(len(list(owners.keys()))-1,len(S)+1)) * diffUtilityCF(all_train_data, target_data, test, owners, A, B, util)
        owners.update({A: A_data})
        owners.update({B: B_data})
    return total

def diffUtilityCF(all_train_data, target_data, test_data, owners, o1, o2, util='decision_tree'):
    return utilityFunc(all_train_data, target_data, test_data, owners[o1], util) - utilityFunc(all_train_data, target_data, test_data, owners[o2],util)

# %%
def utilityFunc(all_train_data, target_data, test_data, data_owner_data, util):
    return utilityLR(target_data, test_data, data_owner_data)


In [ ]:
bc_mini_train = pd.read_csv('bc_mini_train.csv')
bc_mini_test = pd.read_csv('bc_mini_test.csv')

owners = {0: [0, 1, 2, 3], 1: [4, 5, 10, 11], 2: [12, 13, 14, 15, 16, 17, 18], 4:[6, 7, 8, 9]}
print(shapley(bc_mini_train, bc_mini_test, owners, 0))
print(findShapleyMC(bc_mini_train, bc_mini_test, 0, owners, 10000))
print(shapley(bc_mini_train, bc_mini_test, owners, 2) - shapley(bc_mini_train, bc_mini_test, owners, 1))
print(diffShapleyCF(bc_mini_train, bc_mini_train, bc_mini_test, owners, 2, 1, utilityLR))

In [ ]:
reservations_train = pd.read_csv('reservation_train.csv')
reservations_test = pd.read_csv('reservation_test.csv')

owners, category_indices = assignClusters(reservations_train, 'arrival_month')

for i in range(len(owners)):
    # print(len(owners[i]))
    print(category_indices[i])


In [ ]:
def get_key_by_value(dictionary, value):
    for key, val in dictionary.items():
        if val == value:
            return key
    return None
for i in range(len(owners)):
    month = get_key_by_value(category_indices,i+1)
    print(len(owners[month]))
    print(utilityLR(reservations_train, reservations_test, owners[month]))


In [ ]:
for i in range(len(owners)):
    month = get_key_by_value(category_indices,i+1)
    print(findShapleyMC(reservations_train, reservations_test, month, owners, 5000))